## Import data

In [1]:
%matplotlib inline
%reload_ext autoreload
from rtxlib.rtx_run import setup_database, db
from analysis_lib.one_sample_tests import KolmogorovSmirnov
from factorial_anova_analysis import get_raw_data

index = "rtxgaussian-baseline" # cars: 750, smart cars: 750, no pendulum (= rtx50), gaussian with 

results = get_raw_data(index, False, 10000)

data retrieved from file rtxgaussian-baseline-10000.pickle


In [2]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

knobs = {}
data = {}
for r in results:
    knob_str = str(r[1])
    if not knob_str in knobs:        
        knobs[knob_str] = []
    if len(knobs[knob_str]) < 10000:
        knobs[knob_str].append(r[1])
    if not knob_str in data:        
        data[knob_str] = []
    if len(data[knob_str]) < 10000:
        data[knob_str].append(r[0])
        
print "len(knobs):" + str(len(knobs))

for k in knobs:
    print len(knobs[k])
    
print "========="

len(knobs):29
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000


## Ttests with variable sample sizes

In [15]:
from analysis_lib.two_sample_tests import Ttest
from analysis_lib.two_sample_tests import TtestPower
from complaints_generator import generate_complaints
from scipy.stats import binom_test
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc("savefig", dpi=150)
import random 

random.seed(123456)

y_key = 'overhead'
alpha = 0.05
necessary_power = 0.80

fake_run_id = "123456"

def generate_complaints(overheads):
    complaints = []
    for overhead in overheads:
        if overhead > 2.5:
#             complaints.append(1)
            if random.random() > 0.5:
                complaints.append(1)
            else:
                complaints.append(0)
        else:
            complaints.append(0)
    return complaints

knob_values = knobs.keys()
print len(knob_values)
# pp.pprint(knob_values)

best_knob = knob_values[0]

# this_knob_value = knob_values[0]    # or (reverse order): this_knob_value = knob_values[1]
other_knob_values = knob_values[1:] # or (reverse order): other_knob_values = [knob_values[0]]

sample_sizes = []
pvalues = {}
powers = {}

for i in knob_values:
    pvalues[i] = []
    powers[i] = []

interrupted = False
complaints_count = {}
complaints_absolute_threshold = 200
complaints_ratio_threshold = 0.12
complains_alpha = 0.05
        
step = 100

count = 0
for other_knob_value in other_knob_values:   
    for sample_size in range(100, 10000, step):
    #     print "--------"
    #     print "Sample size: " + str(sample_size)
        sample_sizes.append(sample_size)

        analysis_data = {}
        raw_data = data[best_knob] 
#         analysis_data[0] = raw_data[:sample_size] #  also: 
        analysis_data[0] = random.sample(raw_data, sample_size)

        complaints_0 = generate_complaints([d[y_key] for d in analysis_data[0]])
        if best_knob in complaints_count: 
            if complaints_count[best_knob] < sum(complaints_0):
                complaints_count[best_knob] = sum(complaints_0)
        else: 
            complaints_count[best_knob] = sum(complaints_0)
        complaints_0_ratio = sum(complaints_0)/float(len(complaints_0))
    #     print "Complaints for configuration " + knob_name + "=" + str(this_knob_value) + ": " \
    #         + str(sum(complaints_0)) + ", ratio: " + str(complaints_0_ratio)   

        next_complaints_0_no = len(complaints_0) + step
        predicted_complaints_0_no = int(next_complaints_0_no * complaints_0_ratio)
    #     print "At the next step, complaints are expected to be: " + str(predicted_complaints_0_no)
        p_val = binom_test(predicted_complaints_0_no, next_complaints_0_no, complaints_ratio_threshold, alternative="greater")
    #     print p_val
    
        raw_data = data[other_knob_value] 
#         analysis_data[1] = raw_data[:sample_size] #  also: 
        analysis_data[1] = random.sample(raw_data, sample_size)
    
        complaints_1 = generate_complaints([d[y_key] for d in analysis_data[1]])
        if other_knob_value in complaints_count: 
            if complaints_count[other_knob_value] < sum(complaints_1):
                complaints_count[other_knob_value] = sum(complaints_1)
        else: 
            complaints_count[other_knob_value] = sum(complaints_1)
        complaints_1_ratio = sum(complaints_1)/float(len(complaints_1))
#         print "Complaints for configuration " + knob_name + "=" + str(other_knob_value) + ": " \
#             + str(sum(complaints_1)) + ", ratio: " + str(complaints_1_ratio)   

        next_complaints_1_no = len(complaints_1) + step
        predicted_complaints_1_no = int(next_complaints_1_no * complaints_1_ratio)
#         print "At the next step, complaints are expected to be: " + str(predicted_complaints_1_no)
#         print "complaints_1_ratio:" + str(complaints_1_ratio)
        p_val = binom_test(predicted_complaints_1_no, next_complaints_1_no, complaints_ratio_threshold, alternative="greater")
#         print p_val
        if p_val < complains_alpha: 
#             print str(best_knob)
#             print str(other_knob_value)
            print "================"
            print "Sample size: " + str(sample_size)
            print "If we continue, we will have a complaint rate higher than " + str(complaints_ratio_threshold) + \
                " (pvalue: " + str(p_val) + ")"
            print "So we abort here."
            print "================"
            break
        
        t = Ttest(fake_run_id, y_key, alpha).start(analysis_data, {})
        pvalues[other_knob_value].append(t["pvalue"])
        one_sided_p_value = t["pvalue"]/2
        
        if t["statistic"] < 0:
            t_p = TtestPower(fake_run_id, y_key, t["effect_size"], alpha, 'smaller').start(analysis_data, {})            
        else: 
            t_p = TtestPower(fake_run_id, y_key, t["effect_size"], alpha, 'larger').start(analysis_data, {})            
        powers[other_knob_value].append(t_p["power"])
        
#         if one_sided_p_value < alpha and t_p["power"] >= necessary_power and abs(t["mean_diff"]) > min_mean_diff:
        if one_sided_p_value < alpha and t_p["power"] >= necessary_power:
            different = "smaller" if t["statistic"] < 0 else "larger"
            print "================"
            print "Overhead in " 
            pp.pprint(best_knob)
            print " is " + different + " than in "
            pp.pprint(other_knob_value)
            print "sample size: " + str(sample_size)                                 
            print "mean difference: " + str(t["mean_diff"]) 
            print "p value: " + str(one_sided_p_value)                                 
            print "effect size: " + str(t["effect_size"]) 
            print "power: " + str(t_p["power"])
            print "================"
            
            count += 1
            if t["statistic"] >= 0:
                best_knob = other_knob_value
            break


print len(complaints_count)
print "Overall cost (with emergency breaking): " + str(sum(complaints_count.values()))
print str(count) + " out of " + str(len(knob_values)) + " comparisons found significant difference."
print "******************"
print "BEST CONFIGURATION"
print "******************"
pp.pprint(knobs[best_knob][0])


29
Overhead in 
"{u're_route_every_ticks': 70, u'freshness_cut_off_value': 100, u'max_speed_and_length_factor': 2.5, u'exploration_percentage': 0.3, u'freshness_update_factor': 5, u'route_random_sigma': 0.3, u'average_edge_duration_factor': 1.0}"
 is larger than in 
"{u're_route_every_ticks': 10, u'freshness_cut_off_value': 100, u'max_speed_and_length_factor': 2.5, u'exploration_percentage': 0.06175758841698649, u'freshness_update_factor': 20, u'route_random_sigma': 0.0, u'average_edge_duration_factor': 2.5}"
sample size: 2800
mean difference: 0.0766323318469
p value: 0.00245249936893
effect size: 0.0752298794798
power: 0.878929187014
Overhead in 
"{u're_route_every_ticks': 10, u'freshness_cut_off_value': 100, u'max_speed_and_length_factor': 2.5, u'exploration_percentage': 0.06175758841698649, u'freshness_update_factor': 20, u'route_random_sigma': 0.0, u'average_edge_duration_factor': 2.5}"
 is smaller than in 
"{u're_route_every_ticks': 10, u'freshness_cut_off_value': 100, u'max_speed